In [73]:
from __future__ import division
import pandas as pd
import numpy as np
import copy
import pickle
import sys
import gc
pd.options.display.max_columns=1000
pd.options.display.max_rows=1000
from encoding import FreqeuncyEncoding

In [74]:
DATADIR = '../hike/'

In [75]:
users=pd.read_csv(DATADIR + 'user_features.csv')
train=pd.read_csv(DATADIR + 'train.csv',dtype={'is_chat': 'uint8', 'node1_id': 'uint32', 'node2_id': 'uint32'})
test=pd.read_csv(DATADIR + 'test.csv',dtype={'id': 'uint32', 'node1_id': 'uint32', 'node2_id': 'uint32'})

In [15]:
df=pd.concat((train.iloc[:,:2],test.iloc[:,1:3]),axis=0)

In [16]:
df.head()

,node1_id,node2_id
0,8446602,6636127
1,1430102,7433949
2,2803017,8372333
3,4529348,894645
4,5096572,4211638


In [6]:
import pickle

In [7]:
degree = pickle.load(open("degrees_contact.pkl"))

### creating the new_test/train.pkl data

In [8]:
y=train['is_chat']
train.drop('is_chat',axis=1,inplace=True)

In [9]:
degree_df=pd.DataFrame(degree.items(),columns=['node_id','degree'])
degree_df.node_id=degree_df.node_id.astype('uint32')

In [18]:
df=df.merge(degree_df,left_on='node1_id',right_on='node_id',how='left')
df.drop('node_id',axis=1, inplace=True)
df.rename(columns={'degree':'degree_source'},inplace=True)

In [21]:
df=df.merge(degree_df,left_on='node2_id',right_on='node_id',how='left')
df.drop('node_id',axis=1, inplace=True)
df.rename(columns={'degree':'degree_target'},inplace=True)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82438770 entries, 0 to 82438769
Data columns (total 4 columns):
node1_id         uint32
node2_id         uint32
degree_source    float64
degree_target    float64
dtypes: float64(2), uint32(2)
memory usage: 2.5 GB


In [24]:
df.head()

,node1_id,node2_id,degree_source,degree_target
0,8446602,6636127,0.000005,9.680220e-07
1,1430102,7433949,0.000006,1.452033e-06
2,2803017,8372333,0.000006,2.662060e-06
3,4529348,894645,0.000007,8.591195e-06
4,5096572,4211638,0.000008,1.331030e-06


In [25]:
df=df.merge(users,left_on='node1_id',right_on='node_id',how='left')
df.drop('node_id',axis=1,inplace=True)
temp=dict(zip(df.columns[df.columns.str.contains('f')],df.columns[df.columns.str.contains('f')]+'_source'))
df.rename(columns=temp,inplace=True)

In [26]:
gc.collect()

272

In [27]:
df=df.merge(users,left_on='node2_id',right_on='node_id',how='left')
df.drop('node_id',axis=1,inplace=True)
temp=dict(zip(df.columns[df.columns.str.contains('f')],df.columns[df.columns.str.contains('f')]+'_target'))
df.rename(columns=temp,inplace=True)

In [28]:
gc.collect()

25

In [30]:
# df.info()

In [31]:
print df.columns

Index([u'node1_id', u'node2_id', u'degree_source', u'degree_target',
       u'f1_source_target', u'f2_source_target', u'f3_source_target',
       u'f4_source_target', u'f5_source_target', u'f6_source_target',
       u'f7_source_target', u'f8_source_target', u'f9_source_target',
       u'f10_source_target', u'f11_source_target', u'f12_source_target',
       u'f13_source_target', u'f1_target', u'f2_target', u'f3_target',
       u'f4_target', u'f5_target', u'f6_target', u'f7_target', u'f8_target',
       u'f9_target', u'f10_target', u'f11_target', u'f12_target',
       u'f13_target'],
      dtype='object')


In [33]:
train=df.iloc[:train.shape[0],:]
train['is_chat']=y

/home/weightsandbias/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [34]:
test=df.iloc[train.shape[0]:,:]

In [35]:
gc.collect()

0

In [36]:
df.columns

Index([u'node1_id', u'node2_id', u'degree_source', u'degree_target',
       u'f1_source_target', u'f2_source_target', u'f3_source_target',
       u'f4_source_target', u'f5_source_target', u'f6_source_target',
       u'f7_source_target', u'f8_source_target', u'f9_source_target',
       u'f10_source_target', u'f11_source_target', u'f12_source_target',
       u'f13_source_target', u'f1_target', u'f2_target', u'f3_target',
       u'f4_target', u'f5_target', u'f6_target', u'f7_target', u'f8_target',
       u'f9_target', u'f10_target', u'f11_target', u'f12_target',
       u'f13_target'],
      dtype='object')

In [37]:
del df

In [38]:
train.reset_index(drop=True,inplace=True)
test.reset_index(drop=True,inplace=True)

In [39]:
print train.shape, test.shape

(70661802, 31) (11776968, 30)


In [40]:
df=pd.concat((train[['node1_id','node2_id']],test[['node1_id','node2_id']]),axis=0)

In [44]:
fe=FreqeuncyEncoding(categorical_columns=['node1_id','node2_id'],return_df=True,normalize=False)
df=fe.fit_transform(df)

In [45]:
df.node1_id=df.node1_id.astype('uint16')
df.node2_id=df.node2_id.astype('uint16')

In [46]:
train[['node1_id','node2_id']]=df.iloc[:train.shape[0],:]
test[['node1_id','node2_id']]=df.iloc[train.shape[0]:,:]

In [47]:
gc.collect()

0

In [48]:
train['is_chat']=y

In [49]:
train.degree_source= train.degree_source.astype('float32')
train.degree_target= train.degree_target.astype('float32')

test.degree_source= test.degree_source.astype('float32')
test.degree_target= test.degree_target.astype('float32')

In [61]:
for c in [ u'f1_source_target', u'f2_source_target', u'f3_source_target',
       u'f4_source_target', u'f5_source_target', u'f6_source_target',
       u'f7_source_target', u'f8_source_target', u'f9_source_target',
       u'f10_source_target', u'f11_source_target', u'f12_source_target',
       u'f13_source_target', u'f1_target', u'f2_target', u'f3_target',
       u'f4_target', u'f5_target', u'f6_target', u'f7_target', u'f8_target',
       u'f9_target', u'f10_target', u'f11_target', u'f12_target',
       u'f13_target', u'is_chat']:
    train[c] = train[c].astype('uint8')

In [65]:
train.to_pickle('freq_new_train.pkl')

In [70]:
for c in [ u'f1_source_target', u'f2_source_target', u'f3_source_target',
       u'f4_source_target', u'f5_source_target', u'f6_source_target',
       u'f7_source_target', u'f8_source_target', u'f9_source_target',
       u'f10_source_target', u'f11_source_target', u'f12_source_target',
       u'f13_source_target', u'f1_target', u'f2_target', u'f3_target',
       u'f4_target', u'f5_target', u'f6_target', u'f7_target', u'f8_target',
       u'f9_target', u'f10_target', u'f11_target', u'f12_target',
       u'f13_target']:
    test[c] = test[c].astype('uint8')

In [72]:
test.to_pickle('freq_new_test.pkl')